# Ejercicios

Entrenar un modelo de traducción usando el dataset creado.


Enlace al modelo: https://huggingface.co/jumava/mbart-translation

## Instalando librerías

Por defecto, el entorno de Google Colab no tiene instaladas las librerías de [HuggingFace](https://huggingface.co/), por lo que vamos a hacer en primer lugar es instalar las librerías: [Transformers](https://huggingface.co/docs/transformers/index), [Datasets](https://huggingface.co/docs/datasets/index), y [Evaluate](https://huggingface.co/docs/evaluate/index).

In [1]:
!pip install datasets evaluate transformers[sentencepiece] sacrebleu accelerate -Uq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

A continuación nos conectamos al hub de huggingface, lo que nos permitirá subir nuestros modelos a este entorno. Al ejecutar la siguiente celda aparecerá un widget en el cual tendremos que copiar el token generado en el primer paso y pulsar en el botón login.

In [2]:
from huggingface_hub import notebook_login

notebook_login()

## Dataset

Comenzamos descargando el dataset.

In [3]:
from datasets import load_dataset

In [4]:
dataset = load_dataset("jumava/es_en")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/414 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.86k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/2.53k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4 [00:00<?, ? examples/s]

Veamos el contenido de este dataset.

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ES', 'EN'],
        num_rows: 15
    })
    test: Dataset({
        features: ['ES', 'EN'],
        num_rows: 4
    })
})

Podemos ver que tenemos un objeto DatasetDict que puede verse como un diccionario. Dicho diccionario contiene un atributo `train` y un atributo `test`, por lo que en este caso ya disponemos de la división para entrenar nuestro modelo. Vamos a ver alguna de las frases del dataset, para lo que tenemos que acceder al atributo `train`.

In [6]:
dataset['train']

Dataset({
    features: ['ES', 'EN'],
    num_rows: 15
})

Con el anterior comando vemos que tenemos un Dataset con dos columnas: `gendered`, y `neutral`. Si queremos ver el contenido del dataset, lo podemos transformar a formato pandas y verlo como una tabla. Vemos a continuación que cada fila de nuestro dataset contiene una frase y una frase alineada que está neutralizada.

In [7]:
dataset['train'].to_pandas()

,ES,EN
0,* ¿Por qué tienes esa boca tan grande?,"""Why do you have such a big mouth?"""
1,El lobo saltó de la cama y empezó a perseguir ...,The wolf jumped out of bed and started chasing...
2,* Para verte mejor.,"""So I can see you better."""
3,* ¿Por qué tienes esos ojos tan grandes?,"""Why do you have such big eyes?"""
4,* Para olerte mejor.,"""So I can smell you better."""
5,Su madre la mandó que llevara una cesta con du...,Her mother sent her to take a basket of sweets...
6,Había una vez una niña llamada Caperucita Roja.,"Once upon a time, there was a little girl name..."
7,"El lobo llegó antes que Caperucita, entró en l...",The wolf arrived before Little Red Riding Hood...
8,* ¿Por qué tienes esa nariz tan grande?,"""Why do you have such a big nose?"""
9,* ¿Por qué tienes esas orejas tan grandes?,"""Why do you have such big ears?"""


Para poder entrenar un modelo con este dataset es necesario tokenizarlo. Cada modelo tokeniza de una manera distinta, por lo que es necesario indicar el modelo para tokenizar el texto. En nuestro caso vamos a utilizar un modelo llamado [mbart](https://huggingface.co/facebook/mbart-large-50).

In [8]:
from transformers import AutoTokenizer

model_checkpoint = "facebook/mbart-large-50"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

Definimos una función para tokenizar el texto. Notar que para otros datasets
será necesario cambiar el valor de "gendered" y "neutral" por las columnas que queramos
tokenizar, el resto del código no hará falta tocarlo.

In [9]:
max_length = 128


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["ES"],
        max_length=max_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["EN"], max_length=max_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

Tokenizamos el dataset y lo mostramos.

In [10]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['ES', 'EN', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 15
    })
    test: Dataset({
        features: ['ES', 'EN', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4
    })
})

Al igual que ocurría con los modelos de clasificación de texto, podemos ver que han aparecido tres nuevas columnas ('input_ids', 'labels' y 'attention_mask') que serán utilizadas para entrenar el modelo. Notar que ya disponemos de una columna labels que será la que tendrá que predecir el modelo.  

Por último, antes de definir nuestro modelo tenemos que definir una función que se va a encargar de preparar los datos para que sean procesados de manera eficiente por el modelo.

In [11]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer)

## Modelo

Pasamos ahora a definir el modelo, lo primero que vamos a definir son los argumentos con los que vamos a entrenar nuestro modelo.

In [12]:
from transformers import Seq2SeqTrainingArguments


args = Seq2SeqTrainingArguments(
    # Carpeta donde se guardará el modelo
    output_dir="mbart-translation",
    # Evaluar el modelo cada época
    eval_strategy="epoch",
    # Valores para modificar los pesos del modelo
    learning_rate=5.6e-5,
    weight_decay=0.01,
    # Número de ejemplos que se muestran cada vez al modelo
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    # Número de veces que que se muestran todos los datos al modelo
    num_train_epochs=2,
    # Realizar predicciones al validar
    predict_with_generate=True,
    push_to_hub=True,
    report_to="none"
)

A continuación definimos nuestro modelo, para ello usamos la clase `AutoModelForSeq2SeqLM` y vamos a utilizar un modelo pre-entrenado (recordar lo que era el transfer learning). Para ello solo tenemos que indicar el nombre de nuestro modelo (definido previamente en la variable `model_checkpoint`.  

In [13]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

Ahora definimos la función que usaremos para calcular la precisión de nuestro modelo. En este caso usaremos la métrica [sacrebleu](https://huggingface.co/spaces/evaluate-metric/sacrebleu) que nos proporciona la métrica bleu utilizada en el Workshop on Machine Translation (WMT).

In [14]:
import numpy as np

import evaluate

metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result


Ya podemos definir nuestro objeto `trainer` que usaremos para entrenar nuestro modelo (en este caso será un modelo secuencia a secuencia). La estructura de este objeto será siempre la misma. Le tenemos que proporcionar:
1. El modelo.
2. La configuración del entrenamiento.
3. El conjunto de entrenamiento.
4. El conjunto de test.
5. El objeto que prepara los datos.
6. El tokenizador.
7. La métrica.

In [15]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-15-7e05dcb5bafc>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Y ahora entrenamos el modelo mediante el método `train`. Este proceso es algo más lento que el entrenamiento de los modelos de clasificación y puede llevar unos 10 minutos.

In [16]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,4.169657,6.837900,19.500000
2,No log,3.816768,14.492200,26.250000


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=4, training_loss=5.380431175231934, metrics={'train_runtime': 110.0839, 'train_samples_per_second': 0.273, 'train_steps_per_second': 0.036, 'total_flos': 2281412001792.0, 'train_loss': 5.380431175231934, 'epoch': 2.0})

## Compartiendo el modelo

Una vez que tenemos entrenado nuestro modelo, nos interesa compartirlo con el resto del mundo para que puedan usarlo y también compararlo con otros modelos.

Es por ello que vamos a subir nuestro modelo al hub de huggingface. Para ello tenemos que ejecutar el siguiente comando.

In [17]:
%cd mbart-translation
trainer.push_to_hub(commit_message="Training complete", tags="simplification")

/content/mbart-translation


CommitInfo(commit_url='https://huggingface.co/jumava/mbart-translation/commit/74e61e64ad2e0f1d46bf4f258c230c41ddf0b83a', commit_message='Training complete', commit_description='', oid='74e61e64ad2e0f1d46bf4f258c230c41ddf0b83a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jumava/mbart-translation', endpoint='https://huggingface.co', repo_type='model', repo_id='jumava/mbart-translation'), pr_revision=None, pr_num=None)

Al terminar de ejecutarse el comando anterior tendremos nuestro modelo disponible en https://huggingface.co/jumava/mbart-translation.
